In [1]:
"""
testing.ipynb

File for performing testing to implement lottery ticket experiments.

Authors: Jordan Bourdeau, Casey Forey
Date Created: 3/8/24
"""

%load_ext tensorboard
import functools
import numpy as np
import os
import random
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from tensorflow import keras
from keras.callbacks import Callback
from keras import backend as K
from keras import Sequential
from keras.layers import Dense, Input

from tensorflow_model_optimization.sparsity import keras as sparsity
from tensorflow_model_optimization.sparsity.keras import ConstantSparsity, PolynomialDecay, prune_low_magnitude

from src.harness.constants import Constants as C
from src.harness.dataset import download_data, load_and_process_mnist
from src.harness.model import save_model, load_model
from src.harness.pruning import create_pruning_callbacks, create_pruning_parameters
from src.harness.training import train, TrainingRound
from src.lottery_ticket.foundations import paths

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
X_train, Y_train, X_test, Y_test = load_and_process_mnist()


In [4]:
num_epochs: int = 10
batch_size: int = 60
input_shape: tuple = X_train[0].shape
num_classes: int = 10

num_train_samples: int = X_train.shape[0]

end_step: int = np.ceil(1.0 * num_train_samples / batch_size).astype(np.int32) * num_epochs

pruning_parameters: dict = create_pruning_parameters(0.01, 0, end_step, 100)

In [5]:
def count_nonzero_params(model: keras.Model) -> tuple[int, int]:
    weights = model.get_weights()
    total_weights = sum(tf.size(w).numpy() for w in weights)  # Calculate total weights
    nonzero_weights = sum(tf.math.count_nonzero(w).numpy() for w in weights)  # Calculate non-zero weights
    return total_weights, nonzero_weights

def set_seed(random_seed: int):
    # Set seeds for reproducability
    os.environ['PYTHONHASHSEED'] = str(random_seed)
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

# Create a model with the same architecture using all Keras components to check its accuracy with the same parameters
def create_lenet_300_100(input_shape: tuple[int, ...], num_classes: int, optimizer = C.OPTIMIZER) -> keras.Model:
    """
    Function for creating LeNet-300-100 model.

    :param input_shape: Expected input shape for images.
    :param num_classes: Number of potential classes to predict.
    :param optimizer:   Optimizer to use for training.

    :returns: Compiled LeNet-300-100 architecture.
    """
    model = Sequential([
        Input(input_shape),
        Dense(300, activation='relu', kernel_initializer=tf.initializers.GlorotUniform()),
        Dense(100, activation='relu', kernel_initializer=tf.initializers.GlorotUniform()),
        Dense(num_classes, activation='softmax', kernel_initializer=tf.initializers.GlorotUniform()),
    ], name="LeNet-300-100")

    model.compile(
        loss=keras.losses.CategoricalCrossentropy(), 
        optimizer=optimizer(), 
        metrics=['accuracy'])

    return model

def global_pruning_nn(input_shape: tuple[int, ...], num_classes: int, pruning_params: dict) -> keras.Model:
    """
    Function to define the architecture of a neural network model
    following 300 100 architecture for MNIST dataset and using
    provided parameter which are used to prune the model.
    
    :param
    """
    
    model = sparsity.prune_low_magnitude(Sequential([
        Input(input_shape),
        Dense(300, activation='relu', kernel_initializer=tf.initializers.GlorotUniform()),
        Dense(100, activation='relu', kernel_initializer=tf.initializers.GlorotUniform()),
        Dense(num_classes, activation='softmax', kernel_initializer=tf.initializers.GlorotUniform())
    ], name="Pruned_LeNet-300-100"), **pruning_parameters)

    model.compile(
        loss=keras.losses.CategoricalCrossentropy(), 
        optimizer=C.OPTIMIZER(), 
        metrics=['accuracy'])
    
    return model

set_seed(0)
model = global_pruning_nn(input_shape, num_classes, pruning_parameters)
model.summary()

Model: "Pruned_LeNet-300-100"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_dense (  (None, 1, 300)           470702    
 PruneLowMagnitude)                                              
                                                                 
 prune_low_magnitude_dense_1  (None, 1, 100)           60102     
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_dense_2  (None, 1, 10)            2012      
  (PruneLowMagnitude)                                            
                                                                 
Total params: 532,816
Trainable params: 266,610
Non-trainable params: 266,206
_________________________________________________________________


In [6]:
count_nonzero_params(model)

(532816, 532403)

In [7]:
stripped_model: keras.Model = sparsity.strip_pruning(model)
stripped_model.summary()
save_model(stripped_model, 0, 0)

Model: "Pruned_LeNet-300-100"


_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1, 300)            235500    
                                                                 
 dense_1 (Dense)             (None, 1, 100)            30100     
                                                                 
 dense_2 (Dense)             (None, 1, 10)             1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________
Directory 'models/model_0/step_0/model.keras' already exists.


In [8]:
loaded_model: keras.Model = load_model(0, 0)
loaded_model.summary()


Model: "Pruned_LeNet-300-100"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1, 300)            235500    
                                                                 
 dense_1 (Dense)             (None, 1, 100)            30100     
                                                                 
 dense_2 (Dense)             (None, 1, 10)             1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________
